In [1]:
import open3d as o3d
from open3d.web_visualizer import draw
import numpy as np
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [52]:
data_path = "./point_clouds"
pcd_path = "point_cloud_20240806_001539_id_7.npy"
data_dict_1= np.load(os.path.join(data_path, pcd_path), allow_pickle=True).item()["point_cloud"]
pcd_path = "point_cloud_20240806_001540_id_7.npy"
data_dict_2 = np.load(os.path.join(data_path, pcd_path), allow_pickle=True).item()["point_cloud"]

In [53]:
distance_x = np.mean(data_dict_1[:,0]) - np.mean(data_dict_2[:,0])
distance_y = np.mean(data_dict_1[:,1]) - np.mean(data_dict_2[:,1])
distance_z = np.mean(data_dict_1[:,2]) - np.mean(data_dict_2[:,2])

data_dict_1[:, 0] = data_dict_1[:, 0] - distance_x
data_dict_1[:, 1] = data_dict_1[:, 1] - distance_y
data_dict_1[:, 1] = data_dict_1[:, 1] - distance_z

In [54]:
np.mean(data_dict_1[:,0])

-0.35952908270454353

In [55]:
colors_1 = np.zeros_like(data_dict_1)
colors_1[:,1] = 1 + colors_1[:,1]
colors_2 = np.zeros_like(data_dict_2)
colors_2[:,2] = 1 + colors_2[:,2] 

In [56]:
pcd = np.concatenate((data_dict_1, data_dict_2), axis=0)
colors = np.concatenate((colors_1, colors_2), axis=0)

In [57]:
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(pcd)
pcd_o3d.colors = o3d.utility.Vector3dVector(colors)

In [58]:
draw(pcd_o3d)

[Open3D INFO] Window window_7 created.


WebVisualizer(window_uid='window_7')

[Open3D INFO] Sending init frames to window_7.


[653:727][4150639] (stun_port.cc:96): Binding request timed out from 192.168.0.x:44238 (eno1)
[653:727][4150639] (stun_port.cc:96): Binding request timed out from 172.24.249.x:45835 (ztly5r4qjp)
[653:727][4150639] (stun_port.cc:96): Binding request timed out from 10.147.18.x:54575 (ztqu3aetcl)
[653:727][4150639] (stun_port.cc:96): Binding request timed out from 10.43.71.x:40285 (ztvli23lch)
[653:730][4150639] (stun_port.cc:96): Binding request timed out from 10.147.18.x:54575 (ztqu3aetcl)
[653:730][4150639] (stun_port.cc:96): Binding request timed out from 172.24.249.x:45835 (ztly5r4qjp)
[653:730][4150639] (stun_port.cc:96): Binding request timed out from 10.43.71.x:40285 (ztvli23lch)
